- Normalized Difference Vegetation Index: `NDVI = (NIR - Red)/(NIR + Red)`
- Modified Normalized Difference Water Index: `MNDWI = (Green - SWIR1)/(Green + SWIR1)`
- Soil Adjusted Vegetation Index: `SAVI = (1 + L) * ((NIR - Red)/(NIR + Red + L))`

In [ ]:
import os
import matplotlib.pyplot as plt
import pystac_client
from odc.stac import stac_load, configure_s3_access
from odc import stac
import xarray as xr
import rioxarray as rxr
import pandas as pd

In [ ]:
latitude = 27.163
longitude = 82.608
year = 2023

km2deg = 1.0 / 111
x, y = (longitude, latitude)
r = 1 * km2deg
bbox = (x - r, y - r, x + r, y + r)

In [ ]:
catalog = pystac_client.Client.open(
    'https://earth-search.aws.element84.com/v1')

search = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    bbox=bbox,
    datetime=f'{year}',
    query={'eo:cloud_cover': {'lt': 30}, 's2:nodata_pixel_percentage': {'lt': 10}},
    sortby=[{'field': 'properties.eo:cloud_cover', 'direction': 'asc'}]

)
items = search.item_collection()

In [ ]:
ds = stac.load(
    items,
    bands=['red', 'green', 'blue', 'nir', 'swir16'],
    resolution=10,
    chunks={},  # <-- use Dask
    groupby='solar_day',
    preserve_original_order=True
)

In [ ]:
timestamp = pd.to_datetime(items[0].properties['datetime']).tz_convert(None)
scene = ds.sel(time=timestamp)
scene = scene.where(scene != 0)

In [ ]:
offset = -0.1
scale = 0.0001
scene = scale * scene + offset

In [ ]:
scene_da = scene.to_array('band')

In [ ]:
preview = scene_da.rio.reproject(
    scene_da.rio.crs, resolution=300
)

fig, (ax0, ax1) = plt.subplots(1, 2)
fig.set_size_inches(5,5)

preview.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax0,
    robust=True)
ax0.set_title('RGB Visualization')
ax0.set_axis_off()
ax0.set_aspect('equal')

preview.sel(band=['nir', 'red', 'green']).plot.imshow(
    ax=ax1,
    robust=True)
ax1.set_title('NRG Visualization')
ax1.set_axis_off()
plt.show()

### NDVI

In [ ]:
red = scene_da.sel(band='red')
nir = scene_da.sel(band='nir')

nvdi = (nir - red) / (nir + red)

### MNDWI

In [ ]:
green = scene_da.sel(band='green')
swir16 = scene_da.sel(band='swir16')
mndwi = (green - swir16)/(green + swir16)

### SAVI

In [ ]:
savi = 1.5 * ((nir - red) / (nir + red + 0.5))


### Creating Binary Raster with threshold